# Read a file-list and generate shell script commands

In [1]:
# basic packages
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
#import databricks.koalas as ks

# the default `sequence` option merges all partitions into a single catastrophic one : what?
#ks.set_option('compute.default_index_type', 'distributed') 

# plot settings
plt.rc('font', family='serif') 
plt.rc('font', serif='Times New Roman') 
plt.rcParams.update({'font.size': 16})

## Read data 

In [2]:
!pwd

/home/shong/work/gaia/notebook


In [3]:
!ls /home/shong/work/gaia/data/

GaiaSource.header  gaia-csv.list  show3cols.html


In [4]:
with open('/home/shong/work/gaia/data/gaia-csv.list') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 

In [5]:
len(content)

3386

In [6]:
content[:6]

['GaiaSource_000000-003111.csv.gz',
 'GaiaSource_003112-005263.csv.gz',
 'GaiaSource_005264-006601.csv.gz',
 'GaiaSource_006602-007952.csv.gz',
 'GaiaSource_007953-010234.csv.gz',
 'GaiaSource_010235-012597.csv.gz']

In [7]:
content[-1]

'GaiaSource_786097-786431.csv.gz'

In [8]:
def outname (infilename,headstr='',tmodestr='',tailstr='.parquet.snappy'):
    outstr = ''
        

    mcode = infilename.split('.csv')[0]
    outstr = mcode
    
    return headstr+outstr+tailstr

In [9]:
outname(content[0],headstr='')

'GaiaSource_000000-003111.parquet.snappy'

In [10]:
inpath = "hdfs://spark00:54310/common/data/catalog/external-catalogs/External_Catalogs/Gaia_DR3/"

In [11]:
outpath = "hdfs://spark00:54310/common/data/catalog/external-catalogs/parquet/gaia-dr3/raw/"

In [12]:
outname(content[0],headstr=outpath)

'hdfs://spark00:54310/common/data/catalog/external-catalogs/parquet/gaia-dr3/raw/GaiaSource_000000-003111.parquet.snappy'

## Generate the script commands

In [13]:
!pwd

/home/shong/work/gaia/notebook


In [14]:
lencontent = len(content)

In [15]:
print(lencontent)

3386


In [16]:
## temporary assignment for testing
#lencontent = 3

### For L1 and t1 

In [17]:
# write down each command to a shell script 
finalstr = ''
idx=0
while idx < lencontent:
    infilename = content[idx]
    header = 'spark-submit --master spark://spark00:7077 --driver-memory 64g --executor-memory 120g --conf spark.driver.maxResultSize=8g convert-csv-parquet.py '
    body = inpath+infilename+' '+outpath+outname(infilename,headstr='',tmodestr='',tailstr='.parquet.snappy')
    finalstr = finalstr+header+body+'\n'  
    idx=idx+1
    #print(idx)


In [18]:
with open("/home/shong/work/gaia/script/run-convert.sh", "w") as script_file:
    script_file.write(finalstr)

In [19]:
print(finalstr[:3000])

spark-submit --master spark://spark00:7077 --driver-memory 64g --executor-memory 120g --conf spark.driver.maxResultSize=8g convert-csv-parquet.py hdfs://spark00:54310/common/data/catalog/external-catalogs/External_Catalogs/Gaia_DR3/GaiaSource_000000-003111.csv.gz hdfs://spark00:54310/common/data/catalog/external-catalogs/parquet/gaia-dr3/raw/GaiaSource_000000-003111.parquet.snappy
spark-submit --master spark://spark00:7077 --driver-memory 64g --executor-memory 120g --conf spark.driver.maxResultSize=8g convert-csv-parquet.py hdfs://spark00:54310/common/data/catalog/external-catalogs/External_Catalogs/Gaia_DR3/GaiaSource_003112-005263.csv.gz hdfs://spark00:54310/common/data/catalog/external-catalogs/parquet/gaia-dr3/raw/GaiaSource_003112-005263.parquet.snappy
spark-submit --master spark://spark00:7077 --driver-memory 64g --executor-memory 120g --conf spark.driver.maxResultSize=8g convert-csv-parquet.py hdfs://spark00:54310/common/data/catalog/external-catalogs/External_Catalogs/Gaia_DR3/